In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [195]:
dn_response = requests.get("https://www.dn.se/")

In [196]:
dn_html = BeautifulSoup(dn_response.text, "html.parser")

In [200]:
h1_tags = dn_html.find_all("h1")
data = []
skip_headlines = ["Dagens Nyheter"]

for h1 in h1_tags:
    h1_dict = {}
    h1_text = " ".join(h1.text.split())
    
    if h1_text not in skip_headlines:
        data.append(h1_dict)
    
    h1_dict['Headline'] = h1_text

In [203]:
df = pd.DataFrame(headlines)

In [204]:
df

,Headline
0,Märta Stenevi blir nytt språkrör för Miljöpartiet
1,Lockas av makt och inflytande: ”Jag är inte rädd”
2,Ewa Stenberg: Kan MP:s ”Modesty Blaise” skapa ...
3,Tusentals frihetsberövade i nya protester
4,Moskva
...,...
75,"5 favoriter: Film, tv, konst, musik och bok"
76,DN:s ungdomspaket för dig mellan 18 och 25 år
77,Brittiska artisten Sophie död i fallolycka
78,Läs serierna!


In [205]:
teaser_container = dn_html.find_all('div', {'class': 'teaser-package__content'})

master_list = []

for teaser in teaser_container:
    data_dict = {}
    t = teaser.find("div", {"class": "teaser__content"})

    try:
        title = ' '.join(t.h1.text.split())
        title = title.replace('•', '').strip()
        data_dict['Title'] = title
    except:
        pass
    
    try:
        subtitle = ' '.join(t.p.text.split())
        subtitle = subtitle.replace('•', '-').replace('TV | TEXT', '').replace('LIVE', '').strip()
        data_dict['Subtitle'] = subtitle
    except:
        data_dict['Subtitle'] = ''
    
    try:
        link = 'https://www.dn.se'+teaser.find('a')['href']
        data_dict['Link'] = link
    except:
        pass
    master_list.append(data_dict)

print(master_list)

[{'Title': 'Astra Zeneca utökar sina leveranser till EU', 'Subtitle': '”Vi får se om det räcker”, säger Sveriges vaccinsamordnare Richard Bergström.', 'Link': 'https://www.dn.se/sverige/astra-zeneca-utokar-sina-leveranser-till-eu/'}, {'Title': 'Djupa sprickor i partiet – här är arvet efter Trump', 'Subtitle': 'Topprepublikanen Liz Cheney vågade vända sig mot Trump, nu buas hon ut på hemmaplan.', 'Link': 'https://www.dn.se/varlden/jag-arbetar-for-att-vinna-sedan-far-partiet-tid-for-att-laka/'}, {'Title': '5.000 frihetsberövades under protesterna', 'Subtitle': 'Det högsta antalet på en dag i Ryslsland - Julia Navalnaja har släppts ur häktet', 'Link': 'https://www.dn.se/varlden/stora-protester-i-ryssland-vantas-pa-sondagen/'}, {'Title': '”Kändes som om vi hade chansen att vinna”', 'Subtitle': 'Sverige förlorade VM-finalen mot Danmark - ”Målvakten som avgör”.', 'Link': 'https://www.dn.se/sport/live-17-30-sverige-moter-danmark-i-vm-final/'}, {'Title': 'Märta Stenevi blir nytt språkrör för M

In [206]:
df = pd.DataFrame(master_list)

In [210]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
df.to_csv(f'DN_Förstasidan_{date}.csv', index=False)